In [ ]:
!pip install opencv-python mediapipe pygame numpy comtypes pycaw

In [1]:
import cv2
import numpy as np
import pygame
from pygame import mixer
import os
import mediapipe as mp
import time
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize pygame and mixer with better audio settings
pygame.init()
pygame.mixer.init(44100, -16, 2, 2048)
pygame.mixer.set_num_channels(32)

# Define piano dimensions and constants
width = 650
height = 300
white_key_width = width // 10
black_key_width = int(white_key_width * 0.6)
black_key_height = int(height * 0.6)

# Screen dimensions
screen_width = 1280
screen_height = 720

# Calculate piano position
piano_x = (screen_width - width) // 2
piano_y = 20

# Key detection constants
KEY_ACTIVATION_THRESHOLD = 30
KEY_DEBOUNCE_TIME = 0.1
FINGER_HEIGHT_THRESHOLD = 50

class PianoKey:
    def __init__(self):
        self.last_played = 0
        self.is_active = False
        self.sound = None
        self.channel = None
        self.sustain_time = 0.6  # Duration of key sound in seconds

    def play(self, sound):
        current_time = time.time()
        if not self.is_active:
            if self.channel is not None:
                self.channel.stop()
            self.channel = pygame.mixer.find_channel(True)
            if self.channel:
                self.channel.play(sound)
                self.sound = sound
                self.last_played = current_time
                self.is_active = True

    def stop(self):
        if self.is_active:
            if self.channel:
                self.channel.fadeout(200)  # 100ms fadeout
            self.is_active = False

class HandDetector:
    def __init__(self, mode=False, maxHands=2, detectionCon=0.7, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(
            static_image_mode=self.mode,
            max_num_hands=self.maxHands,
            min_detection_confidence=self.detectionCon,
            min_tracking_confidence=self.trackCon
        )
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            if handNo < len(self.results.multi_hand_landmarks):
                myhand = self.results.multi_hand_landmarks[handNo]
                for id, lm in enumerate(myhand.landmark):
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmList.append([id, cx, cy])
                    if draw and id in [4, 8, 12, 16, 20]:
                        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)
        return lmList

def draw_piano_3d(image, white_key_width, black_key_width, black_key_height, highlighted_keys=None):
    if highlighted_keys is None:
        highlighted_keys = []

    # Draw white keys
    for i in range(10):
        x = i * white_key_width
        if i in [key[0] for key in highlighted_keys if not key[1]]:
            gradient_color = (150, 255, 150)
            glow_color = (100, 255, 100, 0.3)
        else:
            gradient_color = (240, 240, 240)
            glow_color = None

        # Draw key body with gradient
        for j in range(height):
            alpha = j / height
            color = tuple(map(lambda x: int(x * (1 - alpha * 0.2)), gradient_color))
            cv2.line(image, (x, j), (x + white_key_width, j), color, 1)

        # Draw 3D effect
        cv2.rectangle(image, (x, 0), (x + 5, height), (180, 180, 180), -1)
        cv2.rectangle(image, (x, 0), (x + white_key_width, height), (0, 0, 0), 2)

        if glow_color:
            overlay = image.copy()
            cv2.rectangle(overlay, (x-5, -5), (x + white_key_width + 5, height + 5),
                         glow_color[:3], -1)
            cv2.addWeighted(overlay, glow_color[3], image, 1 - glow_color[3], 0, image)

    # Draw black keys
    black_key_positions = [1, 2, 4, 5, 6, 8, 9]
    for pos in black_key_positions:
        x = pos * white_key_width - black_key_width // 2
        if x >= 0 and x + black_key_width <= width:
            if pos in [key[0] for key in highlighted_keys if key[1]]:
                key_color = (40, 100, 40)
                glow_color = (30, 100, 30, 0.4)
            else:
                key_color = (0, 0, 0)
                glow_color = None

            cv2.rectangle(image, (x + 2, 2), (x + black_key_width + 2, black_key_height + 2),
                         (20, 20, 20), -1)
            cv2.rectangle(image, (x, 0), (x + black_key_width, black_key_height),
                         key_color, -1)

            if glow_color:
                overlay = image.copy()
                cv2.rectangle(overlay, (x-3, -3), (x + black_key_width + 3, black_key_height + 3),
                             glow_color[:3], -1)
                cv2.addWeighted(overlay, glow_color[3], image, 1 - glow_color[3], 0, image)

def is_finger_in_play_position(finger_y, piano_y):
    return piano_y - FINGER_HEIGHT_THRESHOLD <= finger_y <= piano_y + height

def is_key_press(finger_y, piano_y, is_black):
    if is_black:
        return piano_y <= finger_y <= piano_y + black_key_height
    return piano_y <= finger_y <= piano_y + height

def get_key_index(finger_x, finger_y, piano_x, piano_y):
    if not is_finger_in_play_position(finger_y, piano_y):
        return None, False

    relative_x = finger_x - piano_x
    key_index = int(relative_x // white_key_width)

    if 0 <= key_index < 10:
        if key_index in [1, 2, 4, 5, 6, 8, 9]:
            black_key_x = piano_x + key_index * white_key_width - black_key_width // 2
            if (finger_y < piano_y + black_key_height and
                black_key_x <= finger_x <= black_key_x + black_key_width):
                return key_index, True

        if piano_y <= finger_y <= piano_y + height:
            key_start = piano_x + key_index * white_key_width
            key_end = key_start + white_key_width
            if key_start <= finger_x <= key_end:
                return key_index, False

    return None, False

# Initialize key states
white_key_states = [PianoKey() for _ in range(10)]
black_key_states = [PianoKey() for _ in range(7)]

# Load sound files
sounds_dir = 'tunes'
white_key_files = ['a.wav', 's.wav', 'd.wav', 'f.wav', 'g.wav', 'h.wav', 'j.wav', 'k.wav', 'l.wav', ';.wav']
black_key_files = ['w.wav', 'e.wav', 't.wav', 'y.wav', 'u.wav', 'o.wav', 'p.wav']

white_key_sounds = [mixer.Sound(os.path.join(sounds_dir, file)) for file in white_key_files]
black_key_sounds = [mixer.Sound(os.path.join(sounds_dir, file)) for file in black_key_files]

# Set sound volumes
for sound in white_key_sounds + black_key_sounds:
    sound.set_volume(0.7)

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, screen_width)
cap.set(4, screen_height)

# Initialize hand detector
detector = HandDetector(detectionCon=0.7, maxHands=2)

# Set volume to 100%
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volume.SetMasterVolumeLevel(0.0, None)

# Initialize motion trail variables
trail_points = []
max_trail_length = 10

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    img_with_hands = detector.findHands(frame)
    highlighted_keys = []
    current_time = time.time()

    # Process each hand
    for hand_no in range(2):
        lmList = detector.findPosition(img_with_hands, handNo=hand_no, draw=True)

        if lmList:
            for finger_id in [8, 12]:  # Index and middle fingers
                if finger_id < len(lmList):
                    finger_tip = lmList[finger_id][1:]

                    # Add motion trail
                    trail_points.append((finger_tip[0], finger_tip[1]))
                    if len(trail_points) > max_trail_length:
                        trail_points.pop(0)

                    # Draw motion trail
                    for i in range(1, len(trail_points)):
                        alpha = i / len(trail_points)
                        color = (int(255 * alpha), int(100 * alpha), int(255 * alpha))
                        cv2.line(img_with_hands, trail_points[i-1], trail_points[i], color, 2)

                    # Get key index and handle key press
                    key_index, is_black = get_key_index(
                        finger_tip[0], finger_tip[1], piano_x, piano_y
                    )

                    if key_index is not None:
                        if is_black:
                            black_key_index = [1, 2, 4, 5, 6, 8, 9].index(key_index)
                            key_states = black_key_states
                            key_array_index = black_key_index
                            sounds = black_key_sounds
                        else:
                            key_states = white_key_states
                            key_array_index = key_index
                            sounds = white_key_sounds

                        key_state = key_states[key_array_index]

                        if is_key_press(finger_tip[1], piano_y, is_black):
                            key_state.play(sounds[key_array_index])
                            highlighted_keys.append((key_index, is_black))
                        else:
                            key_state.stop()

    # Stop sounds for inactive keys
    for key_states in [white_key_states, black_key_states]:
        for key_state in key_states:
            if key_state.is_active:
                if current_time - key_state.last_played > key_state.sustain_time:
                    key_state.stop()

    # Create and draw piano
    piano_image = np.ones((height, width, 3), dtype=np.uint8) * 255
    draw_piano_3d(piano_image, white_key_width, black_key_width, black_key_height, highlighted_keys)

    # Overlay piano
    try:
        roi = img_with_hands[piano_y:piano_y + height, piano_x:piano_x + width]
        img_with_hands[piano_y:piano_y + height, piano_x:piano_x + width] = cv2.addWeighted(
            roi, 0.2, piano_image, 0.8, 0
        )
    except ValueError:
        continue

    # Draw play area and text
    cv2.rectangle(img_with_hands,
                 (piano_x, piano_y - FINGER_HEIGHT_THRESHOLD),
                 (piano_x + width, piano_y + height),
                 (0, 255, 0), 2)

    cv2.putText(img_with_hands, "Virtual Piano", (piano_x, piano_y - 30),
                cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img_with_hands, "Press 'q' to quit", (10, screen_height - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    cv2.imshow("Virtual Piano - Two Hands", img_with_hands)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
